<a href="https://colab.research.google.com/github/umedadada/nikkei_software_202407/blob/master/2024_nikkei_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [344]:
# ライブラリの読み込み
import pandas as pd #データを取り扱うためのライブラリ
import plotly.express as px #データを可視化するためのライブラリ

In [345]:
# データの読み込み
raw_df=pd.read_csv('./FEH_00500215_240423175115.csv',encoding="shift-jis")
# データのカラム確認
raw_df.info()
# データの表示
raw_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95040 entries, 0 to 95039
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   cat01_code              95040 non-null  int64 
 1   結果樹面積・10a当たり収量・収穫量・出荷量  95040 non-null  object
 2   cat02_code              95040 non-null  int64 
 3   果樹調査品目                  95040 non-null  object
 4   area_code               95040 non-null  int64 
 5   全国、都府県、全国農業地域           95040 non-null  object
 6   time_code               95040 non-null  int64 
 7   時間軸（年次）                 95040 non-null  object
 8   unit                    95040 non-null  object
 9   value                   95040 non-null  object
dtypes: int64(4), object(6)
memory usage: 7.3+ MB


In [346]:
# データの前処理
df = raw_df[(raw_df['結果樹面積・10a当たり収量・収穫量・出荷量']=='収穫量') & #指標を人口に絞る
            (raw_df['果樹調査品目']=='りんご')] #男女の区別をせず、「総数」を抽出する
df = df[['時間軸（年次）','結果樹面積・10a当たり収量・収穫量・出荷量','果樹調査品目','全国、都府県、全国農業地域','value']] #カラムを絞る
df.columns = ['year','vel_type','type_fruit','area','value'] #カラム名を変更する
df = df.assign(value=df['value'].replace('−',0))# 数値以外の値を除去
df = df.assign(value=df['value'].replace('…',0))# 数値以外の値を除去
df = df.assign(value=df['value'].replace('X',0))# 数値以外の値を除去
df = df.assign(value = df['value'].astype('int64')) #数値カラムの型変換をする
## ソート
df = df.sort_values(by=['year'],ascending=False) #西暦でソート

# df = df.sort_values(by=['year','value'],ascending=False) #西暦でソート
# df

In [347]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# df[df['year']=='2016年']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [348]:
# viz_df = df[(df['area']=='青森県')] #条件の絞り込み
# fig=px.line(viz_df, x='year', y='value') #折れ線を描画
# fig

In [349]:
# 調査年の範囲を調べる
print([min(df['year']),max(df['year'])])

In [ ]:
# 可視化のためのデータ加工
## 条件の絞り込み
viz_df = df[(df['area']!='全国')&
            (df['year']=='2016年')]

## 軸の名前を明示的に指定して可視化
fig = px.pie(viz_df, values='value', names='area',color='area',title='図. 2016年の国内りんご収穫量の内訳')
fig.update_traces(textposition='inside')

fig

In [ ]:
# 可視化のためのデータ加工
## 条件の絞り込み
viz_df = df[(df['area']!='全国')&
            (df['year']=='2016年')]

top5_prefecture = viz_df.nlargest(5,'value')['area']
viz_df.loc[~viz_df['area'].isin(top5_prefecture), 'area'] = 'その他'

## 可視化
fig = px.pie(viz_df, values='value', names='area',color='area',title='図. 2016年の国内りんご収穫量の内訳')
fig.update_traces(textposition='inside')
fig

In [ ]:
## 総収穫量を算出
sum_val=sum(viz_df['value'])
## コンマ表示に整形する
comma_sum_val = "{:,}".format(sum_val)
fig = px.pie(viz_df, values='value', names='area',color='area'
             ,title='図. 2016年の国内りんご収穫量の内訳（総収穫量：{}t ）'.format(comma_sum_val)
             ,color_discrete_map={
              '青森県':'red'
              ,'長野県':'pink'
              ,'山形県':'gray'
              ,'岩手県':'gray'
              ,'福島県':'gray'
              ,'その他':'gray'
              }
             )
fig.update_traces(textinfo='label+percent',textposition='inside')

In [ ]:
# 可視化のためのデータ加工
## 条件の絞り込み
viz_df = df[(df['area']!='全国')&
            (df['year']=='2016年')]

## 総収穫量を算出
sum_val=sum(viz_df['value'])
## コンマ表示に整形する
comma_sum_val = "{:,}".format(sum_val)
# ツリーマップの作成
fig = px.treemap(viz_df, path=['area'], values='value',title='図. 2016年の国内りんご収穫量の内訳（総収穫量：{}t ）'.format(comma_sum_val))
fig.update_traces(textinfo='label+percent root+value')
fig

In [ ]:
# ## ソート
# viz_df = df[(df['area']!='全国')] #条件の絞り込み
# fig=px.bar(viz_df, x='year', y='value') #折れ線を描画
# fig

In [ ]:
#絞り込み
viz_df = df[(df['area']!='全国')] #条件の絞り込み

# 各列の合計を計算
col_sum = viz_df.groupby('year')['value'].sum()

# 各列の合計で各要素を割る
normalized_df = viz_df.copy()  # データを変更せずにコピーする
for year, sum_value in col_sum.items():
    normalized_df.loc[normalized_df['year'] == year, 'value'] /= sum_value

fig=px.bar(normalized_df, x='year', y='value',color='area') #折れ線を描画
fig